<a href="https://colab.research.google.com/github/jagr7/nlp-name-classifier/blob/master/name-classifier-rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP FROM SCRATCH: CLASSIFYING NAMES WITH A CHARACTER-LEVEL RNN

# clone data

In [0]:
!git clone https://github.com/jaybirrd/nlp_name_classifier.git

Cloning into 'nlp_name_classifier'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 52 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (52/52), done.


# prepare the data

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def find_files(path):
  return glob.glob(path)

#print(find_files('nlp_name_classifier/data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicode_to_ascii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
  )

#print(unicode_to_ascii('Ślusàrski'))

category_lines = {}
all_categories = []

def read_lines(filename):
  lines = open(filename, encoding='utf-8').read().strip().split('\n')
  return [unicode_to_ascii(line) for line in lines]

#FUTURE DEV: put a popup here to select different datasets to analyze

for filename in find_files('nlp_name_classifier/data/names/*.txt'):
  category = os.path.splitext(os.path.basename(filename))[0]
  all_categories.append(category)
  lines = read_lines(filename)
  category_lines[category] = lines

n_categories = len(all_categories)

#print(category_lines['Italian'][:5])

# turn names into tensors

In [0]:
import torch

def letter_to_index(letter):
  return all_letters.find(letter)

#turn a letter into a <1 x n_letters> Tensor
def letter_to_tensor(letter):
  tensor = torch.zeros(1,n_letters)
  tensor[0][letter_to_index(letter)] = 1
  return tensor

#turn lines into a <line length x 1 x n_letters> Tensor
def line_to_tensor(line):
  tensor = torch.zeros(len(line),1,n_letters)
  for i, letter in enumerate(line):
    tensor[i][0][letter_to_index(letter)] = 1
    return tensor

#print(line_to_tensor('James').size())
#print(letter_to_tensor('J'))

# creating the network

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
      super(RNN, self).__init__()
      self.hidden_size = hidden_size
      self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
      self.i2o = nn.Linear(input_size + hidden_size, output_size)
      self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
      combined = torch.cat((input, hidden), dim=1)
      hidden = self.i2h(combined)
      output = self.softmax(self.i2o(combined))
      return output, hidden

    def init_hidden(self):
      return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

#FUTURE DEV: set up precomputing of tensors to further optimize

input = line_to_tensor('Albert')
hidden = torch.zeros(1, n_hidden)
output, next_hidden = rnn(input[0], hidden)
print(output, next_hidden)

tensor([[-2.9126, -2.8580, -2.8921, -3.0047, -2.8901, -2.8264, -2.8996, -2.8903,
         -2.9021, -2.9140, -2.8987, -2.9074, -2.8346, -3.0196, -2.8781, -2.8993,
         -2.8394, -2.7860]], grad_fn=<LogSoftmaxBackward>) tensor([[ 0.0689,  0.0792,  0.0035, -0.0706, -0.0394,  0.0248, -0.0952, -0.0113,
          0.1118,  0.0774,  0.0157,  0.0483, -0.1038,  0.0641, -0.0279,  0.0026,
         -0.0588,  0.0736, -0.0022, -0.0117,  0.0119,  0.0232, -0.0739, -0.0377,
         -0.0381,  0.0198, -0.0251, -0.0844, -0.0163,  0.1263,  0.0057, -0.0258,
         -0.0249,  0.0275, -0.1024,  0.0699,  0.1249,  0.1092,  0.0124, -0.0347,
         -0.0942, -0.0052,  0.0924, -0.0724,  0.1068, -0.0882, -0.0428,  0.0693,
          0.0835,  0.0081, -0.0144,  0.0283, -0.0191, -0.0773, -0.0182,  0.0691,
         -0.0493, -0.1274,  0.0900,  0.0489, -0.0414,  0.0372,  0.0447, -0.0258,
          0.0674,  0.1139, -0.0748, -0.0215,  0.0244, -0.0410, -0.0438, -0.1023,
         -0.0168, -0.0574,  0.1142, -0.0934,  0.07

# preparing for training

In [0]:
import random

#netowrk interpreter - guess category based on network output
def category_guess(output):
  n, i = output.topk(1)
  category_index = i.item()
  return all_categories[category_index], category_index

print(category_guess(output))

#generate a training example
def random_choice(x):
  return x[random.randint(0,len(x)-1)]

def random_example():
  r_cat = random_choice(all_categories)
  r_line = random_choice(category_lines[r_cat])
  category_tensor = torch.tensor([all_categories.index(r_cat)], dtype=torch.long)
  line_tensor = line_to_tensor(r_line)
  return r_cat, r_line, category_tensor, line_to_tensor

for i in range(10):
    r_cat, r_line, category_tensor, line_tensor = random_example()
    print('category =', r_cat, '/ line =', r_line)


('Portuguese', 17)
category = Scottish / line = Mccallum
category = Chinese / line = Huang
category = Arabic / line = Tuma
category = Vietnamese / line = Ma
category = Greek / line = Zaloumi
category = Italian / line = Abatescianni
category = Arabic / line = Hakimi
category = Vietnamese / line = Han
category = Spanish / line = Quintana
category = Arabic / line = Almasi


# training the network

In [0]:
#set up the training function
learning_rate = .005

def train(category_tensor, line_tensor):
  hidden = rnn.init_hidden()
  rnn.zero_grad()
  
  for i in range(line_tensor.size()[0]):
    output, hidden = rnn(line_tensor[i], hidden)

  loss = nn.NLLLoss(output, category_tensor)
  loss.backward()

  for p in rnn.parameters():
    p.data.add_(-learning_rate, p.grad.data))

  return output, loss.item()

#train the network
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000

current_loss = 0
all_losses = []

def time(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = start.time()

for i in range(1,n_iters+1):
  r_cat, r_line, category_tensor, line_tensor = random_example()
  output, loss = train(category_tensor, line_tensor)